Notebook for scraping movie grossing data information from Box Office Mojo

The process is split into two functions below. In the first function unique release numbers for each movie are collected. In the second function the actual grossing data for each movie is collected. The collected data represents domestic grossing values for the original release of each movie. 

In [3]:
# Necessary Imports
import pandas as pd
import bs4
from bs4 import BeautifulSoup # BeautifulSoup
import requests
from io import StringIO
import re
import numpy as np
import time
from datetime import datetime

The "retrieve_release_numbers" function is aptly named as it retrieves the release numbers and the years in which the domestic data starts for each movie which was in theaters in and between the input years interval. The release numbers are used to access the each individual movies webpage to obtain its grossing information. These release numbers and years are stored in a text file for use in the second function below. This text file is named for the years interval used. 

In [ ]:
# This function gets the release numbers for every movie shown domestically in any week in the 
# interval of years given.
def retrieve_release_numbers(start_year,end_year):
    print(" ")
    start = time.time()
    # This portions loops through all the weeks in all the years in the interval selected
    # and creates a dictionary of movie release numbers and the years in which the 
    # domestic data for the movie starts
    rl_numbers = {}
    for year in range(start_year,end_year+1):
        print(year)
        response_year = requests.get(url="https://www.boxofficemojo.com/weekly/by-year/"+str(year)+"/")
        if response_year.status_code != 200:
            print("Failed request for year: ",year)
        df = pd.read_html(StringIO(str(BeautifulSoup(response_year.text, 'html.parser').find('table'))))[0]
        weeks = df.loc[df['Top 10 Gross'] != "-"]['Week']
        table_nums = [num+1 for num in weeks.index]
        weeks = [week for week in weeks]
        # Loop through the weeks in a given year
        for i in range(0,len(weeks)):
            if len(str(weeks[i])) > 1:
                response_week = requests.get(url="https://www.boxofficemojo.com/weekly/"+str(year)+
                                             "W"+str(weeks[i])+"/?ref_=bo_wly_table_"+str(table_nums))
            else:
                response_week = requests.get(url="https://www.boxofficemojo.com/weekly/"+str(year)+
                                             "W0"+str(weeks[i])+"/?ref_=bo_wly_table_"+str(table_nums))
            if response_week.status_code != 200:
                print("Failed response for year and week: ",year,weeks[i])
            soup = BeautifulSoup(response_week.text, 'html.parser')
            current_rl_numbers = [str(r).split('/')[2] for r in soup.find('table').find_all('td', attrs={"class":"a-text-left mojo-field-type-release mojo-cell-wide"})]
            for rl_num in current_rl_numbers:
                if rl_num not in rl_numbers:
                    rl_numbers[rl_num] = year
    
    print(" ")
    print("Number of rl numbers collected: ", len(rl_numbers))
    print(" ")

    file1 = open('rawdata/'+str(start_year)+'-'+str(end_year)+'_rl_numbers.txt','w')
    for rl_num in rl_numbers:
        file1.write(str(rl_num)+'\t'+str(rl_numbers[rl_num])+'\n')
    file1.close()

    print("Time to collect all rl numbers:",round((time.time()-start)/60,2))


The "scrape_boxofficemojo_weekly_earnings_webpage" takes each movies release number and goes to the movies webpage and collects the weekly grossing information for that movie. This information is stored in a text file for further cleaning. The years interval inputed designates which file of release numbers should be used for the scraping allow the user to scrape data for movies from specified time periods. 

Release numbers and release year were appended to a movies title to give each movie a unique name in the final dataframe. 

In [ ]:
# This function accepts a start and end year and scrapes together all the grossing data
# by week for every movie released in and between the given years. Only the domestic 
# original release data is collected. 
def scrape_boxofficemojo_weekly_earnings_webpage(start_year,end_year):
    print(" ")
    start = time.time()

    # set up dicitonary to collect all data and turn into dataframe to be saved
    columns = ['Movie Title','Rank','Gross ($)','Theater Number',
               'Change in Theater Number','Gross to Date','Week','Date']
    data_dict = {}
    for column in columns:
        data_dict[column] = []
    
    # get the dictionary of rl numbers
    rl_numbers = {}
    file1 = open('rawdata/'+str(start_year)+'-'+str(end_year)+'_rl_numbers.txt','r')
    A = "Bessie"
    while A != "":
        A = file1.readline()
        if A == "":
            break
        A = A.split('\t')
        rl_numbers[A[0]] = A[1].strip('\n')
    file1.close()
    print("Number of movies:",len(rl_numbers))
    print(" ")
    
    # This portion loops through the rl#'s going to each movies page and obtaining 
    # the data from the page.
    count = 0
    possible_years = [i for i in range(1977,2026)]
    for rl_num in rl_numbers:
        # connect to webpage
        response_movie = requests.get(url="https://www.boxofficemojo.com/release/"+rl_num+"/weekly/?ref_=bo_rl_tab#tabs")
        if response_movie.status_code != 200:
            print("Failed request for movie: ",rl_num)
        soup = BeautifulSoup(response_movie.text, 'html.parser')

        # get movie release year to add to title
        A = [str(x) for x in soup.find_all('div', attrs={'class':'a-section a-spacing-none'}) if "Release Date" in str(x)]
        options = []
        for x in A:
            for i in range(4,len(x)):
                if x[i-4:i].isdigit() == True:
                    options += [int(x[i-4:i])]
        if len(options) < 1:
            print("ERROR: NO RELEASE YEAR FOUND,",re.sub(' - Box Office Mojo','',soup.title.get_text()))
            print(A)
        else:
            release_year = str(np.min(options))
            movie_title = re.sub(' - Box Office Mojo','',soup.title.get_text())+" ("+release_year+")-"+str(rl_num)
            if int(release_year) not in possible_years:
                print("ERROR: RELEASE YEAR NOT IN POSSIBLE YEARS,",movie_title,",",rl_num)
            
            # set current year to the year that the data starts
            current_year = rl_numbers[rl_num]

            # get the information from the webpage into a dataframe and then collect 
            # the desired info and store it in the data dictionary
            df = pd.read_html(StringIO(str(soup.find('table'))))[0]
            movie_title_list = [movie_title for i in range(0,len(df))]
            A = df['Rank']
            rank_list =  []
            for i in range(0,len(A)):
                if A[i] == "-":
                    rank_list += [np.nan]
                else:
                    rank_list += [int(A[i])]
            A = df['Weekly']
            gross_list = []
            for i in range(0,len(A)):
                if A[i] == "-":
                    gross_list += [np.nan]
                else:
                    gross_list += [int(re.sub(',','',A[i]).strip('$'))]
            A = df['Theaters']
            theaters_list = []
            for i in range(0,len(A)):
                if A[i] == "-":
                    theaters_list += [np.nan]
                else:
                    theaters_list += [int(A[i])]
            A = df['Change']
            theater_change_list = []
            for i in range(0,len(A)):
                if A[i] == "-":
                    theater_change_list += [np.nan]
                else:
                    theater_change_list += [int(A[i])]
            A = df['To Date']
            gross_to_date_list = []
            for i in range(0,len(A)):
                if A[i] == "-":
                    gross_to_date_list += [np.nan]
                else:
                    gross_to_date_list += [int(re.sub(',','',A[i]).strip('$'))]
            A = df['Week']
            week_list = []
            for i in range(0,len(A)):
                if A[i] == "-":
                    week_list += [np.nan]
                else:
                    week_list += [int(A[i])]
            A = list(df['Date'])
            dates_list = []
            for i in range(0,len(A)):
                B = A[i].split('-')[0].split(' ')
                if B[0] == "Feb" and B[1] == "29":
                    dates_list += [B[0]+" "+str(int(B[1])-1)]
                else:
                    dates_list += [B[0]+" "+B[1]]
            years_to_append = [current_year]
            for i in range(1,len(dates_list)):
                if (datetime.strptime(dates_list[i],"%b %d")-datetime.strptime(dates_list[i-1],"%b %d")).days < 0:
                    current_year = str(int(current_year)+1)
                if int(current_year) not in possible_years:
                    print("ERROR dates,",movie_title,",",current_year)
                years_to_append += [current_year]
            dates_list_final = []
            for i in range(0,len(dates_list)):
                dates_list_final += [datetime.strptime(dates_list[i]+" "+years_to_append[i],"%b %d %Y")]
            data_dict['Movie Title'] += movie_title_list
            data_dict['Rank'] += rank_list
            data_dict['Gross ($)'] += gross_list
            data_dict['Theater Number'] += theaters_list
            data_dict['Change in Theater Number'] += theater_change_list
            data_dict['Gross to Date'] += gross_to_date_list
            data_dict['Week'] += week_list
            data_dict['Date'] += dates_list_final

            count += 1
            if count%int(len(rl_numbers)/100) == 0:
                print(int((count/len(rl_numbers))*100),"%")


    # This portion creates a pandas dataframe and saves it in a text file
    df_data = pd.DataFrame(data_dict)
    print("number of movies:",len(df_data['Movie Title'].unique()))
    df_data.to_csv('../rawdata/'+str(start_year)+"-"+str(end_year)+".txt",
                   sep = '\t',
                   encoding = 'utf-8',
                   index = False)

    print(" ")
    print("Time to collect data:",round((time.time()-start)/60,3),"minutes")
    print(" ")

BoxOfficeMojo has movie data for years 1977-2025. The total number of unique movies/release numbers was 18003. Of these movies, 8 movies were not included in the final dataset since no release year could be found using the method in the above function. Of these 8, 3 movies grossed over 1 million. Thus, while there was some data loss in the process, we managaged to acquire a large starting dataset.

The total process of scraping all movies from 1977 to 2025 takes around 2 hours. 

In [ ]:
start_year = 1977
end_year = 2025
retrieve_release_numbers(start_year,end_year)
scrape_boxofficemojo_weekly_earnings_webpage(start_year,end_year)


Below is where we examined the data and calculated some summary statistics. We then further cleaned the data and put it into a text file for use in analysis. 

In [2]:
# packages to import
import pandas as pd
import time
import re
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.optimize import curve_fit
import scipy.stats as stats
import statistics as stat
import matplotlib.cm as cm
from prettytable import PrettyTable # type: ignore

Load dataset and stipulate a total gross cutoff value (in millions)

In [ ]:
# stipulate a gross cutoff for the total gross a movie needs to make it into the cleaned dataset. 
# the cutoff is in units ($ millions)
gross_cutoff = 1

df = pd.read_csv("../rawdata/1977-2025.txt",
    sep = '\t')

Examine some summary stats of the data and see what problems need to be tackled when cleaning. It was discovered that the listed weeks on BoxOfficeMojo were in the incorrect order for some movies (as a result of the wrong week number being listed). To remedy this, the 'date' data was used to construct grossing over time information. Additionally, there are some movies with non-unique titles. Final 'gross to date' values and the sum of gross per week values were compared as well to see for which movies these values were not equal. These cases were monitored for different subsets of the data.

In [ ]:
# Show what data is in the data frame
print(" ")
print("Data Columns:")
for column in df.columns:
    print(column)

# Show the number of lines and number of movies in the data
print(" ")
movies = df['Movie Title'].unique()
print("Number of lines:",len(df['Movie Title']))
print("Number of movies:",len(movies))

# Show which movie was in theaters for the most weeks and least weeks
most_weeks = 0
least_weeks = 10000
for movie in movies:
    if len(df.loc[df['Movie Title'] == movie]) > most_weeks:
        most_weeks = len(df.loc[df['Movie Title'] == movie])
        movie_with_most_weeks = movie
    if len(df.loc[df['Movie Title'] == movie]) < least_weeks:
        least_weeks = len(df.loc[df['Movie Title'] == movie])
        movie_with_least_weeks = movie
print(" ")
print("Movie with most weeks in theaters:",movie_with_most_weeks)
print("Number of weeks:",most_weeks)
dates_list = [datetime.strptime(x,"%Y-%m-%d") for x in list(df.loc[df['Movie Title'] == movie_with_most_weeks]['Date'])]
dates_list.sort()
delta = dates_list[-1]-dates_list[0]
print("Final week:",int(delta.days/7)+1)

print("Movie with least weeks in theaters:",movie_with_least_weeks)
print("Number of weeks:",least_weeks)
dates_list = [datetime.strptime(x,"%Y-%m-%d") for x in list(df.loc[df['Movie Title'] == movie_with_least_weeks]['Date'])]
dates_list.sort()
delta = dates_list[-1]-dates_list[0]
print("Final week:",int(delta.days/7)+1)

# Counts the number of movies with the weeks in the wrong order
print(" ")
count = 0
for movie in movies:
    weeks_sorted = list(df.loc[df['Movie Title'] == movie].sort_values(by=['Week'])['Week'])
    weeks = list(df.loc[df['Movie Title'] == movie]['Week'])
    for i in range(0,len(weeks)):
        if weeks[i] != weeks_sorted[i]:
            count += 1
            break
print("total number of movies with weeks in wrong order: ",count)

# counts the number of movies with the dates in the wrong order
print(" ")
print("Movies with the dates out of order:")
count = 0
for movie in movies:
    dates_list = [datetime.strptime(x,"%Y-%m-%d") for x in list(df.loc[df['Movie Title'] == movie]['Date'])]
    for i in range(1,len(dates_list)):
        if (dates_list[i] - dates_list[i-1]).days < 0:
            count += 1
            print(movie)
            break
print("total number of movies with dates in wrong order: ",count)

# see if there are duplicate movie names
print(" ")
print("duplicate movie names: ")
movie_names = {}
for movie in movies:
    movie_name = movie.split('-rl')[0]
    if movie_name in movie_names:
        movie_names[movie_name] += 1
    else:
        movie_names[movie_name] = 1
for movie in movie_names:
    if movie_names[movie] > 1:
        print("duplicate movie name:",movie)

# Shows average total gross and number of movies with a total 
# gross above a certain threshold
print(" ")
print("Movies which have no 'Gross ($)' data:")
count1 = 0
count2 = 0
total_gross_list = []
for movie in movies:
    total_gross = int(df.loc[df['Movie Title'] == movie]['Gross ($)'].sum())
    if total_gross > 1:
        total_gross_list += [total_gross]
    else:
        print("No gross values in 'Gross ($)' column:",movie)
    if (df.loc[df['Movie Title'] == movie]['Gross to Date'].max() > 1) and (total_gross > 1) :
        if total_gross != int(df.loc[df['Movie Title'] == movie]['Gross to Date'].max()):
            count1 += 1
        if abs(total_gross-int(df.loc[df['Movie Title'] == movie]['Gross to Date'].max())) > 100000:
            count2 += 1
print(" ")
print("Number of movies where sum of gross per week does not equal final gross to date:",count1)
print("Number of movies where sum of gross per week and final gross to date differ"+'\n'+"by more than $100,000",count2)
print("Number of movies with > $1 in total gross",len(total_gross_list))
print("Total gross stats are based upon the sum of the gross per week data")
print("Average total gross of these movies:",round(np.average(total_gross_list),0))
print("Average total gross of these movies:",round(np.average(total_gross_list)/1000000,0),"million")
print("Number of movies with total gross above "+str(gross_cutoff)+" million:",len([x for x in total_gross_list if x > gross_cutoff*1000000]))


 
Data Columns:
Movie Title
Rank
Gross ($)
Theater Number
Change in Theater Number
Gross to Date
Week
Date
 
Number of lines: 152499
Number of movies: 17995
Movie with most weeks in theaters: Space Station 3D (2002)-rl3866134017
Number of weeks: 791
Final week: 933
Movie with least weeks in theaters: Close Encounters of the Third Kind (1977)-rl340428289
Number of weeks: 1
Final week: 1
 
total number of movies with weeks in wrong order:  605
 
Movies with the dates iut of order:
total number of movies with dates in wrong order:  0
 
duplicate movie names: 
duplicate movie name: Home (2009)
duplicate movie name: Terkel in Trouble 2010 Re-release (2010)
duplicate movie name: Gold (2017)
 
Movies which have no 'Gross ($)' data:
 
Number of movies where sum of gross per week does not equal final gross to date: 2564
Number of movies where sum of gross per week and final gross to date differ
by more than $100,000 768
Number of movies with > $1 in total gross 17995
Total gross stats are based

Here we examine the same stats as above but for the set of movies which had a total gross above the total gross cutoff

In [6]:
## Calculate the same stats as above but for the subset of movies
## which grossed above the cutoff
print(" ")
print("Gross cutoff:",gross_cutoff*1000000)
print("Gross cutoff:",gross_cutoff,"million")

# get dataset of movies with total gross above cutoff
movies_with_gross_above_cutoff = []
for movie in movies:
    if int(df.loc[df['Movie Title'] == movie]['Gross ($)'].sum()) > (gross_cutoff*1000000):
        movies_with_gross_above_cutoff += [movie]
mask = df['Movie Title'].isin(movies_with_gross_above_cutoff)
df = df[mask]

# Show the number of lines and number of movies in the data
print(" ")
movies = df['Movie Title'].unique()
print("Number of lines:",len(df['Movie Title']))
print("Number of movies:",len(movies))

# Show which movie was in theaters for the most weeks and least weeks
most_weeks = 0
least_weeks = 10000
for movie in movies:
    if len(df.loc[df['Movie Title'] == movie]) > most_weeks:
        most_weeks = len(df.loc[df['Movie Title'] == movie])
        movie_with_most_weeks = movie
    if len(df.loc[df['Movie Title'] == movie]) < least_weeks:
        least_weeks = len(df.loc[df['Movie Title'] == movie])
        movie_with_least_weeks = movie
print(" ")
print("Movie with most weeks in theaters:",movie_with_most_weeks)
print("Number of weeks:",most_weeks)
dates_list = [datetime.strptime(x,"%Y-%m-%d") for x in list(df.loc[df['Movie Title'] == movie_with_most_weeks]['Date'])]
dates_list.sort()
delta = dates_list[-1]-dates_list[0]
print("Final week:",int(delta.days/7)+1)

print("Movie with least weeks in theaters:",movie_with_least_weeks)
print("Number of weeks:",least_weeks)
dates_list = [datetime.strptime(x,"%Y-%m-%d") for x in list(df.loc[df['Movie Title'] == movie_with_least_weeks]['Date'])]
dates_list.sort()
delta = dates_list[-1]-dates_list[0]
print("Final week:",int(delta.days/7)+1)

# Counts the number of movies with the weeks in the wrong order
print(" ")
count = 0
for movie in movies:
    weeks_sorted = list(df.loc[df['Movie Title'] == movie].sort_values(by=['Week'])['Week'])
    weeks = list(df.loc[df['Movie Title'] == movie]['Week'])
    for i in range(0,len(weeks)):
        if weeks[i] != weeks_sorted[i]:
            count += 1
            break
print("total number of movies with weeks in wrong order: ",count)

# Shows average total gross and number of movies with a total 
# gross above a certain threshold
print(" ")
print("Movies which have no 'Gross ($)' data:")
count1 = 0
count2 = 0
total_gross_list = []
for movie in movies:
    total_gross = int(df.loc[df['Movie Title'] == movie]['Gross ($)'].sum())
    if total_gross > 1:
        total_gross_list += [total_gross]
    else:
        print("No gross values in 'Gross ($)' column:",movie)
    if total_gross != int(df.loc[df['Movie Title'] == movie]['Gross to Date'].max()):
        count1 += 1
    if abs(total_gross-int(df.loc[df['Movie Title'] == movie]['Gross to Date'].max())) > 100000:
        count2 += 1
print("Number of movies where sum of gross per week does not equal final gross to date:",count1)
print("Number of movies where sum of gross per week and final gross to date differ"+'\n'+"by more than $100,000",count2)
print("Number of movies with > $1 in total gross",len(total_gross_list))
print("Total gross stats are based upon the sum of the gross per week data")
print("Average total gross of these movies:",round(np.average(total_gross_list),0))
print("Average total gross of these movies:",round(np.average(total_gross_list)/1000000,0),"million")

 
Gross cutoff: 1000000
Gross cutoff: 1 million
 
Number of lines: 89674
Number of movies: 7839
Movie with most weeks in theaters: Space Station 3D (2002)-rl3866134017
Number of weeks: 791
Final week: 933
Movie with least weeks in theaters: Close Encounters of the Third Kind (1977)-rl340428289
Number of weeks: 1
Final week: 1
 
total number of movies with weeks in wrong order:  313
 
Movies which have no 'Gross ($)' data:
Number of movies where sum of gross per week does not equal final gross to date: 784
Number of movies where sum of gross per week and final gross to date differ
by more than $100,000 584
Number of movies with > $1 in total gross 7839
Total gross stats are based upon the sum of the gross per week data to keep in line with time-series analysis
Average total gross of these movies: 38996000.0
Average total gross of these movies: 39.0 million


Finally we filter out movies that had less than 3 weeks of grossing data and calculated summary statistics for this final dataset. This cleaned dataset was saved in a text file fur use in further data analysis. 

In [ ]:
## this portion filters out movies that were in theaters for less than 3 
## weeks and also counts duplicate movie names

# count duplicates
print(" ")
print("duplicate movie names: ")
movie_names = {}
for movie in movies:
    movie_name = movie.split('-rl')[0]
    if movie_name in movie_names:
        movie_names[movie_name] += 1
    else:
        movie_names[movie_name] = 1
duplicate_count = 0
for movie in movie_names:
    if movie_names[movie] > 1:
        print("duplicate movie name:",movie)
        duplicate_count += 1
if duplicate_count == 0:
    print("no duplicate movies in reduced dataset based upon gross cutoff")

# remove movies with less than three weeks in theaters
print(" ")
movies_to_keep = []
for movie in movies:
    if (df.loc[df['Movie Title'] == movie]['Gross ($)'].count()) > 2:
        if len(df.loc[df['Movie Title'] == movie]) != (df.loc[df['Movie Title'] == movie]['Gross ($)'].count()):
            print(movie+" df_subset length != number of values in 'Gross ($)'")
        movies_to_keep += [movie]
mask = df['Movie Title'].isin(movies_to_keep)
df = df[mask]

print(" ")
movies = df['Movie Title'].unique()
print("Number of movies in final dataset:",len(movies))
print("Number of rows in final dataset:",len(df))

# Gross stats for this final dataset
print(" ")
count1 = 0
count2 = 0
total_gross_list = []
for movie in movies:
    total_gross = int(df.loc[df['Movie Title'] == movie]['Gross ($)'].sum())
    if total_gross > 1:
        total_gross_list += [total_gross]
    else:
        print("No gross values in 'Gross ($)' column:",movie)
    if total_gross != int(df.loc[df['Movie Title'] == movie]['Gross to Date'].max()):
        count1 += 1
    if abs(total_gross-int(df.loc[df['Movie Title'] == movie]['Gross to Date'].max())) > 100000:
        count2 += 1
print("Number of movies where sum of gross per week does not equal final gross to date:",count1)
print("Number of movies where sum of gross per week and final gross to date differ"+'\n'+"by more than $100,000",count2)
print("Total gross stats are based upon the sum of the gross per week data to keep in line with time-series analysis")
print("Average total gross of these movies:",round(np.average(total_gross_list),0))
print("Average total gross of these movies:",round(np.average(total_gross_list)/1000000,0),"million")

df = df.drop('Week', axis = 1)
df.to_csv('1977-2025_clean.txt',
                sep = '\t',
                encoding = 'utf-8',
                index = False)


 
duplicate movie names: 
no duplicate movies in reduced dataset based upon gross cutoff
 
 
Number of movies in final dataset: 6960
Number of rows in final dataset: 88322
 
Number of movies where sum of gross per week does not equal final gross to date: 746
Number of movies where sum of gross per week and final gross to date differ
by more than $100,000 548
Total gross stats are based upon the sum of the gross per week data to keep in line with time-series analysis
Average total gross of these movies: 43384440.0
Average total gross of these movies: 43.0 million
